In [1]:
import os
from enum import Enum
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mycolorpy import colorlist as mcp
from z_base import Experiment, Mols, T_ZERO
from scipy.stats import linregress

plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=15)
plt.rcParams["figure.figsize"] = (10, 7)

In [2]:
%matplotlib qt

In [3]:
results_dir = Path(r'Result_data')
dfs=[]
for file in os.listdir(results_dir):
    exp = Experiment.from_hdf5(results_dir/file)
    dfs.append(exp.d)
DF=pd.concat(dfs).reset_index(drop=True)
DF.iloc[:,1:]=DF.iloc[:,1:].map(lambda x: round(x,6))
DF.head(3)

,compound,T,Nu,rho,w_mass,w_vol,x,V,ln_Nu,1_T,D,ln_D
0,butanol,286.15,3.72,0.82668,100.0,100.0,100.0,4.499927,1.313724,0.003495,76.922043,4.342792
1,butanol,286.35,3.70,0.82668,100.0,100.0,100.0,4.475734,1.308333,0.003492,77.391892,4.348882
2,butanol,286.45,3.70,0.82668,100.0,100.0,100.0,4.475734,1.308333,0.003491,77.418919,4.349231


In [4]:
class W(Enum):
    x = "x% mol"
    w_mass = "x% mass"
    w_vol = "w% vol"
    T = 'Temperature [C]'
    Nu = 'Viscosity [cP]'
    ln_Nu = 'log Viscosity'
    V = 'Kinematic viscosity [cCt]'
    ln_D = 'Log Diffusion'
    D = 'Diffusion [nm2/s]'

In [5]:
# Grouped Y(x)
compound = Mols.dmag
x_i = W.T
y_i = W.ln_D
g_i = W.w_mass

compound_filter = (DF["compound"] == compound.name) | (DF["compound"] == Mols.butanol.name)


fig, ax = plt.subplots(num="DT")
grouped = DF[compound_filter].groupby(by=[g_i.name])
colors_list = mcp.gen_color(cmap="turbo", n=len(grouped))
for _, group in grouped:
    y = group[y_i.name]
    T = group[x_i.name]
    color = colors_list.pop()

    ax.scatter(T - T_ZERO, y, c=color, s=2)

    R = linregress(T, y)
    ax.plot(
        np.array([15, 40]),
        R.slope * np.array([15 + T_ZERO, 40 + T_ZERO]) + R.intercept,
        c=color,
        label=f"{group[g_i.name].mean(): 5.2f} %",
    )
ax.set_title(compound.value.upper())
ax.set_xlim(15, 40)
ax.set_xlabel(x_i.value)
ax.set_ylabel(y_i.value)

plt.legend()
plt.show()

In [7]:
# Window Y(x)
compound = Mols.ocm
x_i = W.w_vol
y_i = W.Nu

compound_filter = (DF["compound"] == compound.name)
# | (DF["compound"] == Mols.butanol.name)
grouped = DF[compound_filter]
temper_list = np.arange(15, 41, 5)+T_ZERO
colors_list = mcp.gen_color(cmap="turbo", n=len(temper_list))

fig, ax = plt.subplots(num="DX")
for T in temper_list:
    cond = (T - 1 <= grouped["T"]) & (grouped["T"] <= T + 1)
    group2= grouped[cond]
    group = group2[[x_i.name, y_i.name]].groupby(by=x_i.name).mean().reset_index()

    color = colors_list.pop()
    x = group[x_i.name]
    y = group[y_i.name]
    ax.scatter(x, y, c=color, label=f"{T-T_ZERO: 3.0f} C")

ax.set_title(f"{compound.value.upper()}")
ax.set_xlabel(f"{x_i.value} [%]")
ax.set_ylabel(y_i.value)
plt.legend()
plt.show()

In [35]:
# Density
x_i=W.w_vol
grouped = DF.groupby(by=["compound"])
fig, ax = plt.subplots()
for (compound,*_), group in grouped:
    name = group["compound"].unique()[0]
    group = group[["rho", x_i.name]].groupby(by=[x_i.name]).mean().reset_index()

    x = group[x_i.name]
    y = group["rho"]

    ax.scatter(x, y, label=f"{compound}")
ax.set_title(f"Rho - {x_i.value}")
ax.set_xlabel(f"{x_i.value} [%]")
ax.set_ylabel("Density [g/ml]")
plt.legend()
plt.show()

In [59]:
# E
x_i = W.x
y_i = W.ln_Nu

fig, ax = plt.subplots(num="E")
for (compound, *_), group in DF.groupby(by=["compound"]):
    y = []
    x = []
    for (x_val, *_), sub_group in group.groupby(by=[x_i.name]):
        r = linregress(sub_group[['1_T', y_i.name]])
        y.append(r.slope)
        x.append(x_val)
    group = group[[x_i.name]].groupby(by=[x_i.name]).mean().reset_index()
    ax.plot([x[0],100], [y[0],-2372])
    ax.scatter(x, y, label=f"{compound}")

ax.set_title(f"E {y_i.value}")
ax.set_xlabel(f"{x_i.value} [%]")
ax.set_ylabel("E [kJ]")
plt.legend()
plt.show()